## 스케일링

변수들간의 스케일이 대부분 맞기는 하지만 
어느정도 아웃라이어가 존재하기도 하고 해보기 전까지 모르기 때문에 아래의 4가지 방법을 이용하여 스케일링 하였다

#### 종류 
* standardscaler : 정규분포 이용
* minmaxscaler : 최대/최소값이 각각 0, 1
* maxabsscaler : 최대절대값과 0이 각각 1, 0 이 되도록하는 scaling
* robustscaler : median과 IQR 사용 outlier의 영향을 최소화 한다 

모든 스케일러는 sklearn.preprocessing 안에 각자 이름으로 들어있음

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
print(sys.executable)

C:\Users\ahn18\Anaconda3\python.exe


In [4]:
data = pd.read_csv("./data/ybigta_sdss_train.csv")
test_data = pd.read_csv("./data/ybigta_sdss_test.csv")
data.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,415567,QSO,106,19.20636,18.83202,18.79017,18.36788,18.32136,19.33958,18.72630,...,18.06469,17.18273,16.74307,16.39772,16.28030,17.60089,16.82960,16.46197,16.09962,16.01179
1,733874,QSO,492,18.85419,18.60349,18.48284,18.36558,18.41101,19.17574,18.90761,...,18.90764,18.64323,18.49622,18.38058,18.47802,18.84907,18.59603,18.46161,18.34665,18.40146
2,1009150,QSO,388,19.81733,18.67198,18.46566,18.33873,18.31073,20.17311,19.00520,...,19.85916,18.70682,18.51968,18.37718,18.37598,19.81720,18.66244,18.45800,18.32918,18.31313
3,803041,QSO,531,21.77330,20.19641,20.08064,20.11380,19.83673,22.16908,20.56249,...,21.38145,20.22594,20.13081,20.15601,20.04247,21.76902,20.18548,20.07931,20.10771,19.85110
4,432241,QSO,180,22.53572,20.07427,19.23110,18.94043,18.65541,22.82620,20.41057,...,22.09349,20.14050,19.25699,18.98682,18.63438,22.58755,20.07051,19.21149,18.90649,18.63881


스케이링 하는 변수에 id, type ,fiberid는 제거해주었다.

In [5]:
data_for_scaling = data[data.columns[3:]]
data_for_scaling.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,19.20636,18.83202,18.79017,18.36788,18.32136,19.33958,18.72630,18.42127,18.05413,18.00387,18.06469,17.18273,16.74307,16.39772,16.28030,17.60089,16.82960,16.46197,16.09962,16.01179
1,18.85419,18.60349,18.48284,18.36558,18.41101,19.17574,18.90761,18.79145,18.68332,18.75945,18.90764,18.64323,18.49622,18.38058,18.47802,18.84907,18.59603,18.46161,18.34665,18.40146
2,19.81733,18.67198,18.46566,18.33873,18.31073,20.17311,19.00520,18.82004,18.69565,18.66105,19.85916,18.70682,18.51968,18.37718,18.37598,19.81720,18.66244,18.45800,18.32918,18.31313
3,21.77330,20.19641,20.08064,20.11380,19.83673,22.16908,20.56249,20.47989,20.49381,20.34039,21.38145,20.22594,20.13081,20.15601,20.04247,21.76902,20.18548,20.07931,20.10771,19.85110
4,22.53572,20.07427,19.23110,18.94043,18.65541,22.82620,20.41057,19.56960,19.29224,18.98282,22.09349,20.14050,19.25699,18.98682,18.63438,22.58755,20.07051,19.21149,18.90649,18.63881


* 4가지 스케일링이 거의 비슷한 방식으로 진행되기 때문에 현재 데이터에 맞는 함수를 작성하였다

In [25]:
def scaling_func(df, scaler) :
    '''
    param : dataframe / scaler object
    return : scaled dataframe / fitting scaler
    '''
    scaler = scaler()
    # type과 id를 제외하고 학습
    data_for_scaling = df.drop(['id', 'type', 'fiberID'], axis = 1)
    scaler.fit(data_for_scaling)
    # 학습후 변환
    train_scaled = scaler.transform(data_for_scaling)
    # 학습후 변환한 데이터를 다시 원래 데이터로 만들기
    result = pd.DataFrame(train_scaled, columns = data_for_scaling.columns)
    result = pd.concat([df[['id','type', "fiberID"]], result], axis=1)
    return result, scaler

### Standard Scaler 사용

In [26]:
from sklearn.preprocessing import StandardScaler
std_data, std_scaler = scaling_func(data, StandardScaler)
std_data.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,415567,QSO,106,-0.654723,-0.314537,0.007668,-0.017116,0.121432,-0.730518,-0.518475,...,-1.051998,-1.046129,-1.073464,-1.121079,-1.023651,-1.310344,-1.241693,-1.227162,-1.290683,-1.160810
1,733874,QSO,492,-0.838003,-0.449605,-0.202817,-0.018833,0.186924,-0.820256,-0.409097,...,-0.628527,-0.225724,0.037374,0.224665,0.406708,-0.667661,-0.223549,0.051361,0.247088,0.420959
2,1009150,QSO,388,-0.336754,-0.409125,-0.214584,-0.038876,0.113666,-0.273982,-0.350225,...,-0.150515,-0.190004,0.052239,0.222357,0.340297,-0.169174,-0.185271,0.049053,0.235133,0.362491
3,803041,QSO,531,0.681195,0.491859,0.891489,1.286220,1.228457,0.819239,0.589226,...,0.614233,0.663330,1.073090,1.429628,1.424911,0.835812,0.692586,1.085680,1.452283,1.380503
4,432241,QSO,180,1.077983,0.419671,0.309653,0.410295,0.365466,1.179153,0.497579,...,0.971938,0.615336,0.519416,0.636113,0.508473,1.257270,0.626319,0.530817,0.630219,0.578065


### minmax Scaler 사용

In [27]:
from sklearn.preprocessing import MinMaxScaler
mm_data, mm_scaler = scaling_func(data, MinMaxScaler)
mm_data.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,415567,QSO,106,0.537382,0.529359,0.530178,0.510288,0.538282,0.365205,0.423735,...,0.258892,0.243564,0.293435,0.238532,0.272473,0.448843,0.383797,0.385904,0.381903,0.389974
1,733874,QSO,492,0.513202,0.512405,0.506850,0.510118,0.545584,0.355816,0.437155,...,0.291514,0.308397,0.381436,0.318452,0.372198,0.541821,0.517053,0.529825,0.539107,0.555502
2,1009150,QSO,388,0.579332,0.517486,0.505546,0.508134,0.537417,0.412969,0.444379,...,0.328338,0.311220,0.382613,0.318315,0.367568,0.613938,0.522063,0.529565,0.537884,0.549383
3,803041,QSO,531,0.713633,0.630577,0.628131,0.639307,0.661695,0.527344,0.559649,...,0.387251,0.378655,0.463485,0.390012,0.443187,0.759331,0.636958,0.646256,0.662311,0.655915
4,432241,QSO,180,0.765982,0.621516,0.563647,0.552598,0.565488,0.564999,0.548404,...,0.414806,0.374862,0.419623,0.342887,0.379293,0.820304,0.628285,0.583796,0.578273,0.571942


### maxabs Scaler 사용

In [29]:
from sklearn.preprocessing import MaxAbsScaler
ma_data, ma_scaler = scaling_func(data, MaxAbsScaler)
ma_data.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,415567,QSO,106,0.740301,0.748011,0.752216,0.734868,0.763685,0.635806,0.706344,...,0.485417,0.502081,0.543267,0.464654,0.503825,0.704040,0.673239,0.658631,0.645675,0.645154
1,733874,QSO,492,0.726726,0.738934,0.739913,0.734776,0.767422,0.630419,0.713183,...,0.508068,0.544757,0.600152,0.520842,0.571837,0.753967,0.743902,0.738635,0.735792,0.741439
2,1009150,QSO,388,0.763850,0.741654,0.739225,0.733702,0.763242,0.663209,0.716864,...,0.533636,0.546615,0.600913,0.520745,0.568679,0.792692,0.746559,0.738490,0.735092,0.737880
3,803041,QSO,531,0.839242,0.802205,0.803876,0.804720,0.826850,0.728828,0.775604,...,0.574542,0.591003,0.653190,0.571151,0.620252,0.870766,0.807486,0.803358,0.806420,0.799849
4,432241,QSO,180,0.868629,0.797354,0.769867,0.757775,0.777609,0.750431,0.769873,...,0.593675,0.588507,0.624837,0.538020,0.576676,0.903507,0.802887,0.768637,0.758245,0.751003


### robust Scaler 사용

In [30]:
from sklearn.preprocessing import RobustScaler
rb_data, rb_scaler= scaling_func(data, RobustScaler)
rb_data.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,415567,QSO,106,-0.435893,-0.291574,-0.061425,-0.111254,0.045898,-0.526436,-0.469058,...,-0.878988,-0.808048,-0.702073,-0.717412,-0.654057,-1.076404,-0.950766,-0.802411,-0.819421,-0.727297
1,733874,QSO,492,-0.580244,-0.413752,-0.220812,-0.112515,0.093583,-0.599909,-0.368806,...,-0.453638,-0.124812,0.072741,0.148958,0.295791,-0.463942,-0.125875,0.075270,0.155184,0.285476
2,1009150,QSO,388,-0.185463,-0.377135,-0.229722,-0.127228,0.040244,-0.152645,-0.314845,...,0.026496,-0.095064,0.083110,0.147472,0.251690,0.011104,-0.094863,0.073686,0.147607,0.248041
3,803041,QSO,531,0.616269,0.437863,0.607836,0.845503,0.851924,0.742433,0.546231,...,0.794639,0.615596,0.795158,0.924695,0.971942,0.968832,0.616370,0.785311,0.919010,0.899853
4,432241,QSO,180,0.928778,0.372564,0.167249,0.202501,0.223579,1.037113,0.462229,...,1.153932,0.575626,0.408968,0.413842,0.363370,1.370472,0.562681,0.404408,0.398004,0.386068


In [11]:
import catboost as cb
cb_model_std = cb.CatBoostClassifier()
cb_model_mm = cb.CatBoostClassifier()
cb_model_ma = cb.CatBoostClassifier()
cb_model_rb = cb.CatBoostClassifier()

In [12]:
X_std = std_data[data.columns[2:]]
X_mm = mm_data[data.columns[2:]]
X_ma = ma_data[data.columns[2:]]
X_rb = rb_data[data.columns[2:]]

y = data['type']

In [13]:
cb_model_std.fit(X_std, y)

0:	learn: 2.6926061	total: 362ms	remaining: 6m 1s
1:	learn: 2.5202741	total: 592ms	remaining: 4m 55s
2:	learn: 2.3874659	total: 820ms	remaining: 4m 32s
3:	learn: 2.2759944	total: 1.05s	remaining: 4m 20s
4:	learn: 2.1808330	total: 1.28s	remaining: 4m 14s
5:	learn: 2.1018191	total: 1.5s	remaining: 4m 9s
6:	learn: 2.0292334	total: 1.74s	remaining: 4m 6s
7:	learn: 1.9664018	total: 1.97s	remaining: 4m 3s
8:	learn: 1.9070229	total: 2.19s	remaining: 4m 1s
9:	learn: 1.8539631	total: 2.42s	remaining: 4m
10:	learn: 1.8056311	total: 2.66s	remaining: 3m 59s
11:	learn: 1.7610348	total: 2.89s	remaining: 3m 58s
12:	learn: 1.7203762	total: 3.12s	remaining: 3m 56s
13:	learn: 1.6810912	total: 3.35s	remaining: 3m 55s
14:	learn: 1.6462434	total: 3.58s	remaining: 3m 55s
15:	learn: 1.6127006	total: 3.8s	remaining: 3m 53s
16:	learn: 1.5813686	total: 4.03s	remaining: 3m 53s
17:	learn: 1.5521009	total: 4.26s	remaining: 3m 52s
18:	learn: 1.5242098	total: 4.49s	remaining: 3m 51s
19:	learn: 1.4974558	total: 4.72s

158:	learn: 0.7056424	total: 36s	remaining: 3m 10s
159:	learn: 0.7035312	total: 36.2s	remaining: 3m 10s
160:	learn: 0.7028174	total: 36.4s	remaining: 3m 9s
161:	learn: 0.7018386	total: 36.6s	remaining: 3m 9s
162:	learn: 0.7007755	total: 36.9s	remaining: 3m 9s
163:	learn: 0.6991323	total: 37.1s	remaining: 3m 9s
164:	learn: 0.6977699	total: 37.3s	remaining: 3m 8s
165:	learn: 0.6964583	total: 37.5s	remaining: 3m 8s
166:	learn: 0.6944045	total: 37.8s	remaining: 3m 8s
167:	learn: 0.6930004	total: 38s	remaining: 3m 8s
168:	learn: 0.6923678	total: 38.2s	remaining: 3m 7s
169:	learn: 0.6908246	total: 38.4s	remaining: 3m 7s
170:	learn: 0.6895657	total: 38.7s	remaining: 3m 7s
171:	learn: 0.6889565	total: 38.9s	remaining: 3m 7s
172:	learn: 0.6883252	total: 39.1s	remaining: 3m 6s
173:	learn: 0.6866889	total: 39.3s	remaining: 3m 6s
174:	learn: 0.6855205	total: 39.5s	remaining: 3m 6s
175:	learn: 0.6846985	total: 39.8s	remaining: 3m 6s
176:	learn: 0.6839130	total: 40s	remaining: 3m 5s
177:	learn: 0.68

315:	learn: 0.5859316	total: 1m 10s	remaining: 2m 33s
316:	learn: 0.5855986	total: 1m 11s	remaining: 2m 33s
317:	learn: 0.5851621	total: 1m 11s	remaining: 2m 33s
318:	learn: 0.5844929	total: 1m 11s	remaining: 2m 32s
319:	learn: 0.5840274	total: 1m 11s	remaining: 2m 32s
320:	learn: 0.5835821	total: 1m 12s	remaining: 2m 32s
321:	learn: 0.5831612	total: 1m 12s	remaining: 2m 32s
322:	learn: 0.5828091	total: 1m 12s	remaining: 2m 32s
323:	learn: 0.5826347	total: 1m 12s	remaining: 2m 31s
324:	learn: 0.5823956	total: 1m 13s	remaining: 2m 31s
325:	learn: 0.5818225	total: 1m 13s	remaining: 2m 31s
326:	learn: 0.5814535	total: 1m 13s	remaining: 2m 31s
327:	learn: 0.5811330	total: 1m 13s	remaining: 2m 30s
328:	learn: 0.5804552	total: 1m 13s	remaining: 2m 30s
329:	learn: 0.5799821	total: 1m 14s	remaining: 2m 30s
330:	learn: 0.5798264	total: 1m 14s	remaining: 2m 30s
331:	learn: 0.5796619	total: 1m 14s	remaining: 2m 30s
332:	learn: 0.5790001	total: 1m 14s	remaining: 2m 29s
333:	learn: 0.5786566	total:

468:	learn: 0.5349825	total: 1m 45s	remaining: 1m 59s
469:	learn: 0.5348650	total: 1m 45s	remaining: 1m 59s
470:	learn: 0.5345365	total: 1m 45s	remaining: 1m 58s
471:	learn: 0.5342611	total: 1m 45s	remaining: 1m 58s
472:	learn: 0.5341924	total: 1m 46s	remaining: 1m 58s
473:	learn: 0.5339054	total: 1m 46s	remaining: 1m 58s
474:	learn: 0.5337247	total: 1m 46s	remaining: 1m 57s
475:	learn: 0.5335812	total: 1m 46s	remaining: 1m 57s
476:	learn: 0.5334045	total: 1m 47s	remaining: 1m 57s
477:	learn: 0.5332452	total: 1m 47s	remaining: 1m 57s
478:	learn: 0.5329767	total: 1m 47s	remaining: 1m 56s
479:	learn: 0.5327849	total: 1m 47s	remaining: 1m 56s
480:	learn: 0.5324490	total: 1m 48s	remaining: 1m 56s
481:	learn: 0.5323277	total: 1m 48s	remaining: 1m 56s
482:	learn: 0.5320008	total: 1m 48s	remaining: 1m 56s
483:	learn: 0.5317377	total: 1m 48s	remaining: 1m 55s
484:	learn: 0.5313989	total: 1m 48s	remaining: 1m 55s
485:	learn: 0.5312999	total: 1m 49s	remaining: 1m 55s
486:	learn: 0.5310106	total:

621:	learn: 0.5046613	total: 2m 19s	remaining: 1m 24s
622:	learn: 0.5044601	total: 2m 19s	remaining: 1m 24s
623:	learn: 0.5042479	total: 2m 20s	remaining: 1m 24s
624:	learn: 0.5040263	total: 2m 20s	remaining: 1m 24s
625:	learn: 0.5039416	total: 2m 20s	remaining: 1m 23s
626:	learn: 0.5038295	total: 2m 20s	remaining: 1m 23s
627:	learn: 0.5037950	total: 2m 20s	remaining: 1m 23s
628:	learn: 0.5036981	total: 2m 21s	remaining: 1m 23s
629:	learn: 0.5036385	total: 2m 21s	remaining: 1m 23s
630:	learn: 0.5035196	total: 2m 21s	remaining: 1m 22s
631:	learn: 0.5034268	total: 2m 21s	remaining: 1m 22s
632:	learn: 0.5032761	total: 2m 22s	remaining: 1m 22s
633:	learn: 0.5030513	total: 2m 22s	remaining: 1m 22s
634:	learn: 0.5029726	total: 2m 22s	remaining: 1m 21s
635:	learn: 0.5027998	total: 2m 22s	remaining: 1m 21s
636:	learn: 0.5025954	total: 2m 22s	remaining: 1m 21s
637:	learn: 0.5024412	total: 2m 23s	remaining: 1m 21s
638:	learn: 0.5022277	total: 2m 23s	remaining: 1m 21s
639:	learn: 0.5021102	total:

775:	learn: 0.4844636	total: 2m 54s	remaining: 50.2s
776:	learn: 0.4843816	total: 2m 54s	remaining: 50s
777:	learn: 0.4842880	total: 2m 54s	remaining: 49.8s
778:	learn: 0.4841908	total: 2m 54s	remaining: 49.6s
779:	learn: 0.4839999	total: 2m 54s	remaining: 49.3s
780:	learn: 0.4839210	total: 2m 55s	remaining: 49.1s
781:	learn: 0.4837999	total: 2m 55s	remaining: 48.9s
782:	learn: 0.4836835	total: 2m 55s	remaining: 48.7s
783:	learn: 0.4835093	total: 2m 55s	remaining: 48.4s
784:	learn: 0.4833411	total: 2m 56s	remaining: 48.2s
785:	learn: 0.4832595	total: 2m 56s	remaining: 48s
786:	learn: 0.4831897	total: 2m 56s	remaining: 47.8s
787:	learn: 0.4831119	total: 2m 56s	remaining: 47.5s
788:	learn: 0.4829805	total: 2m 56s	remaining: 47.3s
789:	learn: 0.4828267	total: 2m 57s	remaining: 47.1s
790:	learn: 0.4826581	total: 2m 57s	remaining: 46.9s
791:	learn: 0.4826146	total: 2m 57s	remaining: 46.6s
792:	learn: 0.4825040	total: 2m 57s	remaining: 46.4s
793:	learn: 0.4822995	total: 2m 58s	remaining: 46.

932:	learn: 0.4687614	total: 3m 29s	remaining: 15s
933:	learn: 0.4685957	total: 3m 29s	remaining: 14.8s
934:	learn: 0.4684594	total: 3m 29s	remaining: 14.6s
935:	learn: 0.4683146	total: 3m 29s	remaining: 14.3s
936:	learn: 0.4682441	total: 3m 30s	remaining: 14.1s
937:	learn: 0.4681234	total: 3m 30s	remaining: 13.9s
938:	learn: 0.4680817	total: 3m 30s	remaining: 13.7s
939:	learn: 0.4679319	total: 3m 30s	remaining: 13.5s
940:	learn: 0.4678109	total: 3m 30s	remaining: 13.2s
941:	learn: 0.4677759	total: 3m 31s	remaining: 13s
942:	learn: 0.4677278	total: 3m 31s	remaining: 12.8s
943:	learn: 0.4675893	total: 3m 31s	remaining: 12.6s
944:	learn: 0.4674935	total: 3m 31s	remaining: 12.3s
945:	learn: 0.4673834	total: 3m 32s	remaining: 12.1s
946:	learn: 0.4672595	total: 3m 32s	remaining: 11.9s
947:	learn: 0.4671701	total: 3m 32s	remaining: 11.7s
948:	learn: 0.4671379	total: 3m 32s	remaining: 11.4s
949:	learn: 0.4671074	total: 3m 32s	remaining: 11.2s
950:	learn: 0.4669984	total: 3m 33s	remaining: 11s

In [14]:
cb_model_mm.fit(X_mm, y)

0:	learn: 2.6926061	total: 222ms	remaining: 3m 42s
1:	learn: 2.5202741	total: 444ms	remaining: 3m 41s
2:	learn: 2.3874659	total: 666ms	remaining: 3m 41s
3:	learn: 2.2759944	total: 889ms	remaining: 3m 41s
4:	learn: 2.1808330	total: 1.13s	remaining: 3m 44s
5:	learn: 2.1018191	total: 1.36s	remaining: 3m 45s
6:	learn: 2.0292334	total: 1.6s	remaining: 3m 46s
7:	learn: 1.9664018	total: 1.83s	remaining: 3m 46s
8:	learn: 1.9070229	total: 2.06s	remaining: 3m 46s
9:	learn: 1.8539631	total: 2.29s	remaining: 3m 47s
10:	learn: 1.8056311	total: 2.52s	remaining: 3m 46s
11:	learn: 1.7610348	total: 2.75s	remaining: 3m 46s
12:	learn: 1.7203762	total: 2.97s	remaining: 3m 45s
13:	learn: 1.6810912	total: 3.2s	remaining: 3m 45s
14:	learn: 1.6462434	total: 3.43s	remaining: 3m 45s
15:	learn: 1.6127006	total: 3.65s	remaining: 3m 44s
16:	learn: 1.5813686	total: 3.89s	remaining: 3m 44s
17:	learn: 1.5521009	total: 4.11s	remaining: 3m 44s
18:	learn: 1.5242098	total: 4.35s	remaining: 3m 44s
19:	learn: 1.4974558	tot

158:	learn: 0.7056424	total: 35.8s	remaining: 3m 9s
159:	learn: 0.7035312	total: 36s	remaining: 3m 8s
160:	learn: 0.7028174	total: 36.2s	remaining: 3m 8s
161:	learn: 0.7018386	total: 36.4s	remaining: 3m 8s
162:	learn: 0.7007755	total: 36.6s	remaining: 3m 8s
163:	learn: 0.6991323	total: 36.9s	remaining: 3m 7s
164:	learn: 0.6977699	total: 37.1s	remaining: 3m 7s
165:	learn: 0.6964583	total: 37.3s	remaining: 3m 7s
166:	learn: 0.6944045	total: 37.5s	remaining: 3m 7s
167:	learn: 0.6930004	total: 37.8s	remaining: 3m 6s
168:	learn: 0.6923678	total: 38s	remaining: 3m 6s
169:	learn: 0.6908246	total: 38.2s	remaining: 3m 6s
170:	learn: 0.6895657	total: 38.4s	remaining: 3m 6s
171:	learn: 0.6889565	total: 38.7s	remaining: 3m 6s
172:	learn: 0.6883252	total: 38.9s	remaining: 3m 5s
173:	learn: 0.6866889	total: 39.1s	remaining: 3m 5s
174:	learn: 0.6855205	total: 39.3s	remaining: 3m 5s
175:	learn: 0.6846985	total: 39.5s	remaining: 3m 5s
176:	learn: 0.6839130	total: 39.8s	remaining: 3m 4s
177:	learn: 0.68

315:	learn: 0.5859316	total: 1m 10s	remaining: 2m 33s
316:	learn: 0.5855986	total: 1m 10s	remaining: 2m 32s
317:	learn: 0.5851621	total: 1m 11s	remaining: 2m 32s
318:	learn: 0.5844929	total: 1m 11s	remaining: 2m 32s
319:	learn: 0.5840274	total: 1m 11s	remaining: 2m 32s
320:	learn: 0.5835821	total: 1m 11s	remaining: 2m 32s
321:	learn: 0.5831612	total: 1m 12s	remaining: 2m 31s
322:	learn: 0.5828091	total: 1m 12s	remaining: 2m 31s
323:	learn: 0.5826347	total: 1m 12s	remaining: 2m 31s
324:	learn: 0.5823956	total: 1m 12s	remaining: 2m 31s
325:	learn: 0.5818225	total: 1m 13s	remaining: 2m 30s
326:	learn: 0.5814535	total: 1m 13s	remaining: 2m 30s
327:	learn: 0.5811330	total: 1m 13s	remaining: 2m 30s
328:	learn: 0.5804552	total: 1m 13s	remaining: 2m 30s
329:	learn: 0.5799821	total: 1m 13s	remaining: 2m 30s
330:	learn: 0.5798264	total: 1m 14s	remaining: 2m 29s
331:	learn: 0.5796619	total: 1m 14s	remaining: 2m 29s
332:	learn: 0.5790001	total: 1m 14s	remaining: 2m 29s
333:	learn: 0.5786566	total:

468:	learn: 0.5349825	total: 1m 45s	remaining: 1m 58s
469:	learn: 0.5348650	total: 1m 45s	remaining: 1m 58s
470:	learn: 0.5345365	total: 1m 45s	remaining: 1m 58s
471:	learn: 0.5342611	total: 1m 45s	remaining: 1m 58s
472:	learn: 0.5341924	total: 1m 45s	remaining: 1m 58s
473:	learn: 0.5339054	total: 1m 46s	remaining: 1m 57s
474:	learn: 0.5337247	total: 1m 46s	remaining: 1m 57s
475:	learn: 0.5335812	total: 1m 46s	remaining: 1m 57s
476:	learn: 0.5334045	total: 1m 46s	remaining: 1m 57s
477:	learn: 0.5332452	total: 1m 47s	remaining: 1m 56s
478:	learn: 0.5329767	total: 1m 47s	remaining: 1m 56s
479:	learn: 0.5327849	total: 1m 47s	remaining: 1m 56s
480:	learn: 0.5324490	total: 1m 47s	remaining: 1m 56s
481:	learn: 0.5323277	total: 1m 47s	remaining: 1m 56s
482:	learn: 0.5320008	total: 1m 48s	remaining: 1m 55s
483:	learn: 0.5317377	total: 1m 48s	remaining: 1m 55s
484:	learn: 0.5313989	total: 1m 48s	remaining: 1m 55s
485:	learn: 0.5312999	total: 1m 48s	remaining: 1m 55s
486:	learn: 0.5310106	total:

621:	learn: 0.5046613	total: 2m 19s	remaining: 1m 24s
622:	learn: 0.5044601	total: 2m 19s	remaining: 1m 24s
623:	learn: 0.5042479	total: 2m 19s	remaining: 1m 24s
624:	learn: 0.5040263	total: 2m 19s	remaining: 1m 23s
625:	learn: 0.5039416	total: 2m 20s	remaining: 1m 23s
626:	learn: 0.5038295	total: 2m 20s	remaining: 1m 23s
627:	learn: 0.5037950	total: 2m 20s	remaining: 1m 23s
628:	learn: 0.5036981	total: 2m 20s	remaining: 1m 23s
629:	learn: 0.5036385	total: 2m 21s	remaining: 1m 22s
630:	learn: 0.5035196	total: 2m 21s	remaining: 1m 22s
631:	learn: 0.5034268	total: 2m 21s	remaining: 1m 22s
632:	learn: 0.5032761	total: 2m 21s	remaining: 1m 22s
633:	learn: 0.5030513	total: 2m 22s	remaining: 1m 21s
634:	learn: 0.5029726	total: 2m 22s	remaining: 1m 21s
635:	learn: 0.5027998	total: 2m 22s	remaining: 1m 21s
636:	learn: 0.5025954	total: 2m 22s	remaining: 1m 21s
637:	learn: 0.5024412	total: 2m 22s	remaining: 1m 21s
638:	learn: 0.5022277	total: 2m 23s	remaining: 1m 20s
639:	learn: 0.5021102	total:

775:	learn: 0.4844636	total: 2m 53s	remaining: 50.2s
776:	learn: 0.4843816	total: 2m 54s	remaining: 50s
777:	learn: 0.4842880	total: 2m 54s	remaining: 49.7s
778:	learn: 0.4841908	total: 2m 54s	remaining: 49.5s
779:	learn: 0.4839999	total: 2m 54s	remaining: 49.3s
780:	learn: 0.4839210	total: 2m 54s	remaining: 49.1s
781:	learn: 0.4837999	total: 2m 55s	remaining: 48.8s
782:	learn: 0.4836835	total: 2m 55s	remaining: 48.6s
783:	learn: 0.4835093	total: 2m 55s	remaining: 48.4s
784:	learn: 0.4833411	total: 2m 55s	remaining: 48.2s
785:	learn: 0.4832595	total: 2m 56s	remaining: 47.9s
786:	learn: 0.4831897	total: 2m 56s	remaining: 47.7s
787:	learn: 0.4831119	total: 2m 56s	remaining: 47.5s
788:	learn: 0.4829805	total: 2m 56s	remaining: 47.3s
789:	learn: 0.4828267	total: 2m 56s	remaining: 47s
790:	learn: 0.4826581	total: 2m 57s	remaining: 46.8s
791:	learn: 0.4826146	total: 2m 57s	remaining: 46.6s
792:	learn: 0.4825040	total: 2m 57s	remaining: 46.4s
793:	learn: 0.4822995	total: 2m 57s	remaining: 46.

932:	learn: 0.4687614	total: 3m 28s	remaining: 15s
933:	learn: 0.4685957	total: 3m 29s	remaining: 14.8s
934:	learn: 0.4684594	total: 3m 29s	remaining: 14.6s
935:	learn: 0.4683146	total: 3m 29s	remaining: 14.3s
936:	learn: 0.4682441	total: 3m 29s	remaining: 14.1s
937:	learn: 0.4681234	total: 3m 30s	remaining: 13.9s
938:	learn: 0.4680817	total: 3m 30s	remaining: 13.7s
939:	learn: 0.4679319	total: 3m 30s	remaining: 13.4s
940:	learn: 0.4678109	total: 3m 30s	remaining: 13.2s
941:	learn: 0.4677759	total: 3m 30s	remaining: 13s
942:	learn: 0.4677278	total: 3m 31s	remaining: 12.8s
943:	learn: 0.4675893	total: 3m 31s	remaining: 12.5s
944:	learn: 0.4674935	total: 3m 31s	remaining: 12.3s
945:	learn: 0.4673834	total: 3m 31s	remaining: 12.1s
946:	learn: 0.4672595	total: 3m 32s	remaining: 11.9s
947:	learn: 0.4671701	total: 3m 32s	remaining: 11.6s
948:	learn: 0.4671379	total: 3m 32s	remaining: 11.4s
949:	learn: 0.4671074	total: 3m 32s	remaining: 11.2s
950:	learn: 0.4669984	total: 3m 32s	remaining: 11s

In [15]:
cb_model_ma.fit(X_ma, y)

0:	learn: 2.6926061	total: 224ms	remaining: 3m 43s
1:	learn: 2.5202741	total: 446ms	remaining: 3m 42s
2:	learn: 2.3874659	total: 667ms	remaining: 3m 41s
3:	learn: 2.2759944	total: 885ms	remaining: 3m 40s
4:	learn: 2.1808330	total: 1.1s	remaining: 3m 39s
5:	learn: 2.1018191	total: 1.33s	remaining: 3m 40s
6:	learn: 2.0292334	total: 1.56s	remaining: 3m 41s
7:	learn: 1.9664018	total: 1.78s	remaining: 3m 40s
8:	learn: 1.9070229	total: 2s	remaining: 3m 40s
9:	learn: 1.8539631	total: 2.23s	remaining: 3m 40s
10:	learn: 1.8056311	total: 2.45s	remaining: 3m 40s
11:	learn: 1.7610348	total: 2.67s	remaining: 3m 40s
12:	learn: 1.7203762	total: 2.9s	remaining: 3m 40s
13:	learn: 1.6810912	total: 3.12s	remaining: 3m 39s
14:	learn: 1.6462434	total: 3.34s	remaining: 3m 39s
15:	learn: 1.6127006	total: 3.57s	remaining: 3m 39s
16:	learn: 1.5813686	total: 3.8s	remaining: 3m 39s
17:	learn: 1.5521009	total: 4.02s	remaining: 3m 39s
18:	learn: 1.5242098	total: 4.24s	remaining: 3m 39s
19:	learn: 1.4974558	total: 

158:	learn: 0.7056424	total: 35.8s	remaining: 3m 9s
159:	learn: 0.7035312	total: 36s	remaining: 3m 8s
160:	learn: 0.7028174	total: 36.2s	remaining: 3m 8s
161:	learn: 0.7018386	total: 36.4s	remaining: 3m 8s
162:	learn: 0.7007755	total: 36.7s	remaining: 3m 8s
163:	learn: 0.6991323	total: 36.9s	remaining: 3m 8s
164:	learn: 0.6977699	total: 37.1s	remaining: 3m 7s
165:	learn: 0.6964583	total: 37.3s	remaining: 3m 7s
166:	learn: 0.6944045	total: 37.6s	remaining: 3m 7s
167:	learn: 0.6930004	total: 37.8s	remaining: 3m 7s
168:	learn: 0.6923678	total: 38s	remaining: 3m 6s
169:	learn: 0.6908246	total: 38.2s	remaining: 3m 6s
170:	learn: 0.6895657	total: 38.5s	remaining: 3m 6s
171:	learn: 0.6889565	total: 38.7s	remaining: 3m 6s
172:	learn: 0.6883252	total: 38.9s	remaining: 3m 6s
173:	learn: 0.6866889	total: 39.2s	remaining: 3m 5s
174:	learn: 0.6855205	total: 39.4s	remaining: 3m 5s
175:	learn: 0.6846985	total: 39.6s	remaining: 3m 5s
176:	learn: 0.6839130	total: 39.8s	remaining: 3m 5s
177:	learn: 0.68

315:	learn: 0.5859316	total: 1m 11s	remaining: 2m 34s
316:	learn: 0.5855986	total: 1m 11s	remaining: 2m 34s
317:	learn: 0.5851621	total: 1m 11s	remaining: 2m 34s
318:	learn: 0.5844929	total: 1m 12s	remaining: 2m 34s
319:	learn: 0.5840274	total: 1m 12s	remaining: 2m 33s
320:	learn: 0.5835821	total: 1m 12s	remaining: 2m 33s
321:	learn: 0.5831612	total: 1m 12s	remaining: 2m 33s
322:	learn: 0.5828091	total: 1m 13s	remaining: 2m 33s
323:	learn: 0.5826347	total: 1m 13s	remaining: 2m 32s
324:	learn: 0.5823956	total: 1m 13s	remaining: 2m 32s
325:	learn: 0.5818225	total: 1m 13s	remaining: 2m 32s
326:	learn: 0.5814535	total: 1m 13s	remaining: 2m 32s
327:	learn: 0.5811330	total: 1m 14s	remaining: 2m 32s
328:	learn: 0.5804552	total: 1m 14s	remaining: 2m 31s
329:	learn: 0.5799821	total: 1m 14s	remaining: 2m 31s
330:	learn: 0.5798264	total: 1m 14s	remaining: 2m 31s
331:	learn: 0.5796619	total: 1m 15s	remaining: 2m 31s
332:	learn: 0.5790001	total: 1m 15s	remaining: 2m 30s
333:	learn: 0.5786566	total:

468:	learn: 0.5349825	total: 1m 45s	remaining: 1m 59s
469:	learn: 0.5348650	total: 1m 46s	remaining: 1m 59s
470:	learn: 0.5345365	total: 1m 46s	remaining: 1m 59s
471:	learn: 0.5342611	total: 1m 46s	remaining: 1m 59s
472:	learn: 0.5341924	total: 1m 46s	remaining: 1m 59s
473:	learn: 0.5339054	total: 1m 47s	remaining: 1m 58s
474:	learn: 0.5337247	total: 1m 47s	remaining: 1m 58s
475:	learn: 0.5335812	total: 1m 47s	remaining: 1m 58s
476:	learn: 0.5334045	total: 1m 47s	remaining: 1m 58s
477:	learn: 0.5332452	total: 1m 47s	remaining: 1m 57s
478:	learn: 0.5329767	total: 1m 48s	remaining: 1m 57s
479:	learn: 0.5327849	total: 1m 48s	remaining: 1m 57s
480:	learn: 0.5324490	total: 1m 48s	remaining: 1m 57s
481:	learn: 0.5323277	total: 1m 48s	remaining: 1m 56s
482:	learn: 0.5320008	total: 1m 49s	remaining: 1m 56s
483:	learn: 0.5317377	total: 1m 49s	remaining: 1m 56s
484:	learn: 0.5313989	total: 1m 49s	remaining: 1m 56s
485:	learn: 0.5312999	total: 1m 49s	remaining: 1m 56s
486:	learn: 0.5310106	total:

621:	learn: 0.5046613	total: 2m 20s	remaining: 1m 25s
622:	learn: 0.5044601	total: 2m 21s	remaining: 1m 25s
623:	learn: 0.5042479	total: 2m 21s	remaining: 1m 25s
624:	learn: 0.5040263	total: 2m 21s	remaining: 1m 24s
625:	learn: 0.5039416	total: 2m 21s	remaining: 1m 24s
626:	learn: 0.5038295	total: 2m 21s	remaining: 1m 24s
627:	learn: 0.5037950	total: 2m 22s	remaining: 1m 24s
628:	learn: 0.5036981	total: 2m 22s	remaining: 1m 24s
629:	learn: 0.5036385	total: 2m 22s	remaining: 1m 23s
630:	learn: 0.5035196	total: 2m 22s	remaining: 1m 23s
631:	learn: 0.5034268	total: 2m 23s	remaining: 1m 23s
632:	learn: 0.5032761	total: 2m 23s	remaining: 1m 23s
633:	learn: 0.5030513	total: 2m 23s	remaining: 1m 22s
634:	learn: 0.5029726	total: 2m 23s	remaining: 1m 22s
635:	learn: 0.5027998	total: 2m 23s	remaining: 1m 22s
636:	learn: 0.5025954	total: 2m 24s	remaining: 1m 22s
637:	learn: 0.5024412	total: 2m 24s	remaining: 1m 21s
638:	learn: 0.5022277	total: 2m 24s	remaining: 1m 21s
639:	learn: 0.5021102	total:

775:	learn: 0.4844636	total: 2m 55s	remaining: 50.6s
776:	learn: 0.4843816	total: 2m 55s	remaining: 50.4s
777:	learn: 0.4842880	total: 2m 55s	remaining: 50.2s
778:	learn: 0.4841908	total: 2m 56s	remaining: 49.9s
779:	learn: 0.4839999	total: 2m 56s	remaining: 49.7s
780:	learn: 0.4839210	total: 2m 56s	remaining: 49.5s
781:	learn: 0.4837999	total: 2m 56s	remaining: 49.3s
782:	learn: 0.4836835	total: 2m 56s	remaining: 49s
783:	learn: 0.4835093	total: 2m 57s	remaining: 48.8s
784:	learn: 0.4833411	total: 2m 57s	remaining: 48.6s
785:	learn: 0.4832595	total: 2m 57s	remaining: 48.4s
786:	learn: 0.4831897	total: 2m 57s	remaining: 48.1s
787:	learn: 0.4831119	total: 2m 58s	remaining: 47.9s
788:	learn: 0.4829805	total: 2m 58s	remaining: 47.7s
789:	learn: 0.4828267	total: 2m 58s	remaining: 47.5s
790:	learn: 0.4826581	total: 2m 58s	remaining: 47.2s
791:	learn: 0.4826146	total: 2m 58s	remaining: 47s
792:	learn: 0.4825040	total: 2m 59s	remaining: 46.8s
793:	learn: 0.4822995	total: 2m 59s	remaining: 46.

932:	learn: 0.4687614	total: 3m 30s	remaining: 15.1s
933:	learn: 0.4685957	total: 3m 30s	remaining: 14.9s
934:	learn: 0.4684594	total: 3m 31s	remaining: 14.7s
935:	learn: 0.4683146	total: 3m 31s	remaining: 14.4s
936:	learn: 0.4682441	total: 3m 31s	remaining: 14.2s
937:	learn: 0.4681234	total: 3m 31s	remaining: 14s
938:	learn: 0.4680817	total: 3m 31s	remaining: 13.8s
939:	learn: 0.4679319	total: 3m 32s	remaining: 13.5s
940:	learn: 0.4678109	total: 3m 32s	remaining: 13.3s
941:	learn: 0.4677759	total: 3m 32s	remaining: 13.1s
942:	learn: 0.4677278	total: 3m 32s	remaining: 12.9s
943:	learn: 0.4675893	total: 3m 33s	remaining: 12.6s
944:	learn: 0.4674935	total: 3m 33s	remaining: 12.4s
945:	learn: 0.4673834	total: 3m 33s	remaining: 12.2s
946:	learn: 0.4672595	total: 3m 33s	remaining: 12s
947:	learn: 0.4671701	total: 3m 33s	remaining: 11.7s
948:	learn: 0.4671379	total: 3m 34s	remaining: 11.5s
949:	learn: 0.4671074	total: 3m 34s	remaining: 11.3s
950:	learn: 0.4669984	total: 3m 34s	remaining: 11.

In [16]:
cb_model_rb.fit(X_rb, y)

0:	learn: 2.6926061	total: 223ms	remaining: 3m 43s
1:	learn: 2.5202741	total: 445ms	remaining: 3m 42s
2:	learn: 2.3874659	total: 672ms	remaining: 3m 43s
3:	learn: 2.2759944	total: 898ms	remaining: 3m 43s
4:	learn: 2.1808330	total: 1.13s	remaining: 3m 43s
5:	learn: 2.1018191	total: 1.36s	remaining: 3m 44s
6:	learn: 2.0292334	total: 1.59s	remaining: 3m 45s
7:	learn: 1.9664018	total: 1.81s	remaining: 3m 44s
8:	learn: 1.9070229	total: 2.03s	remaining: 3m 43s
9:	learn: 1.8539631	total: 2.26s	remaining: 3m 43s
10:	learn: 1.8056311	total: 2.48s	remaining: 3m 43s
11:	learn: 1.7610348	total: 2.71s	remaining: 3m 43s
12:	learn: 1.7203762	total: 2.94s	remaining: 3m 43s
13:	learn: 1.6810912	total: 3.16s	remaining: 3m 42s
14:	learn: 1.6462434	total: 3.38s	remaining: 3m 41s
15:	learn: 1.6127006	total: 3.6s	remaining: 3m 41s
16:	learn: 1.5813686	total: 3.82s	remaining: 3m 40s
17:	learn: 1.5521009	total: 4.04s	remaining: 3m 40s
18:	learn: 1.5242098	total: 4.27s	remaining: 3m 40s
19:	learn: 1.4974558	to

158:	learn: 0.7056424	total: 35.5s	remaining: 3m 7s
159:	learn: 0.7035312	total: 35.7s	remaining: 3m 7s
160:	learn: 0.7028174	total: 35.9s	remaining: 3m 7s
161:	learn: 0.7018386	total: 36.1s	remaining: 3m 6s
162:	learn: 0.7007755	total: 36.4s	remaining: 3m 6s
163:	learn: 0.6991323	total: 36.6s	remaining: 3m 6s
164:	learn: 0.6977699	total: 36.8s	remaining: 3m 6s
165:	learn: 0.6964583	total: 37s	remaining: 3m 5s
166:	learn: 0.6944045	total: 37.2s	remaining: 3m 5s
167:	learn: 0.6930004	total: 37.5s	remaining: 3m 5s
168:	learn: 0.6923678	total: 37.7s	remaining: 3m 5s
169:	learn: 0.6908246	total: 37.9s	remaining: 3m 5s
170:	learn: 0.6895657	total: 38.1s	remaining: 3m 4s
171:	learn: 0.6889565	total: 38.4s	remaining: 3m 4s
172:	learn: 0.6883252	total: 38.6s	remaining: 3m 4s
173:	learn: 0.6866889	total: 38.8s	remaining: 3m 4s
174:	learn: 0.6855205	total: 39s	remaining: 3m 4s
175:	learn: 0.6846985	total: 39.3s	remaining: 3m 3s
176:	learn: 0.6839130	total: 39.5s	remaining: 3m 3s
177:	learn: 0.68

315:	learn: 0.5859316	total: 1m 10s	remaining: 2m 33s
316:	learn: 0.5855986	total: 1m 10s	remaining: 2m 32s
317:	learn: 0.5851621	total: 1m 11s	remaining: 2m 32s
318:	learn: 0.5844929	total: 1m 11s	remaining: 2m 32s
319:	learn: 0.5840274	total: 1m 11s	remaining: 2m 32s
320:	learn: 0.5835821	total: 1m 11s	remaining: 2m 32s
321:	learn: 0.5831612	total: 1m 12s	remaining: 2m 31s
322:	learn: 0.5828091	total: 1m 12s	remaining: 2m 31s
323:	learn: 0.5826347	total: 1m 12s	remaining: 2m 31s
324:	learn: 0.5823956	total: 1m 12s	remaining: 2m 31s
325:	learn: 0.5818225	total: 1m 13s	remaining: 2m 30s
326:	learn: 0.5814535	total: 1m 13s	remaining: 2m 30s
327:	learn: 0.5811330	total: 1m 13s	remaining: 2m 30s
328:	learn: 0.5804552	total: 1m 13s	remaining: 2m 30s
329:	learn: 0.5799821	total: 1m 13s	remaining: 2m 30s
330:	learn: 0.5798264	total: 1m 14s	remaining: 2m 29s
331:	learn: 0.5796619	total: 1m 14s	remaining: 2m 29s
332:	learn: 0.5790001	total: 1m 14s	remaining: 2m 29s
333:	learn: 0.5786566	total:

468:	learn: 0.5349825	total: 1m 45s	remaining: 1m 59s
469:	learn: 0.5348650	total: 1m 45s	remaining: 1m 59s
470:	learn: 0.5345365	total: 1m 45s	remaining: 1m 58s
471:	learn: 0.5342611	total: 1m 45s	remaining: 1m 58s
472:	learn: 0.5341924	total: 1m 46s	remaining: 1m 58s
473:	learn: 0.5339054	total: 1m 46s	remaining: 1m 58s
474:	learn: 0.5337247	total: 1m 46s	remaining: 1m 57s
475:	learn: 0.5335812	total: 1m 46s	remaining: 1m 57s
476:	learn: 0.5334045	total: 1m 47s	remaining: 1m 57s
477:	learn: 0.5332452	total: 1m 47s	remaining: 1m 57s
478:	learn: 0.5329767	total: 1m 47s	remaining: 1m 56s
479:	learn: 0.5327849	total: 1m 47s	remaining: 1m 56s
480:	learn: 0.5324490	total: 1m 48s	remaining: 1m 56s
481:	learn: 0.5323277	total: 1m 48s	remaining: 1m 56s
482:	learn: 0.5320008	total: 1m 48s	remaining: 1m 56s
483:	learn: 0.5317377	total: 1m 48s	remaining: 1m 55s
484:	learn: 0.5313989	total: 1m 48s	remaining: 1m 55s
485:	learn: 0.5312999	total: 1m 49s	remaining: 1m 55s
486:	learn: 0.5310106	total:

621:	learn: 0.5046613	total: 2m 19s	remaining: 1m 24s
622:	learn: 0.5044601	total: 2m 19s	remaining: 1m 24s
623:	learn: 0.5042479	total: 2m 20s	remaining: 1m 24s
624:	learn: 0.5040263	total: 2m 20s	remaining: 1m 24s
625:	learn: 0.5039416	total: 2m 20s	remaining: 1m 23s
626:	learn: 0.5038295	total: 2m 20s	remaining: 1m 23s
627:	learn: 0.5037950	total: 2m 20s	remaining: 1m 23s
628:	learn: 0.5036981	total: 2m 21s	remaining: 1m 23s
629:	learn: 0.5036385	total: 2m 21s	remaining: 1m 23s
630:	learn: 0.5035196	total: 2m 21s	remaining: 1m 22s
631:	learn: 0.5034268	total: 2m 21s	remaining: 1m 22s
632:	learn: 0.5032761	total: 2m 22s	remaining: 1m 22s
633:	learn: 0.5030513	total: 2m 22s	remaining: 1m 22s
634:	learn: 0.5029726	total: 2m 22s	remaining: 1m 21s
635:	learn: 0.5027998	total: 2m 22s	remaining: 1m 21s
636:	learn: 0.5025954	total: 2m 22s	remaining: 1m 21s
637:	learn: 0.5024412	total: 2m 23s	remaining: 1m 21s
638:	learn: 0.5022277	total: 2m 23s	remaining: 1m 21s
639:	learn: 0.5021102	total:

775:	learn: 0.4844636	total: 2m 54s	remaining: 50.3s
776:	learn: 0.4843816	total: 2m 54s	remaining: 50.1s
777:	learn: 0.4842880	total: 2m 54s	remaining: 49.8s
778:	learn: 0.4841908	total: 2m 54s	remaining: 49.6s
779:	learn: 0.4839999	total: 2m 55s	remaining: 49.4s
780:	learn: 0.4839210	total: 2m 55s	remaining: 49.2s
781:	learn: 0.4837999	total: 2m 55s	remaining: 48.9s
782:	learn: 0.4836835	total: 2m 55s	remaining: 48.7s
783:	learn: 0.4835093	total: 2m 56s	remaining: 48.5s
784:	learn: 0.4833411	total: 2m 56s	remaining: 48.3s
785:	learn: 0.4832595	total: 2m 56s	remaining: 48s
786:	learn: 0.4831897	total: 2m 56s	remaining: 47.8s
787:	learn: 0.4831119	total: 2m 56s	remaining: 47.6s
788:	learn: 0.4829805	total: 2m 57s	remaining: 47.4s
789:	learn: 0.4828267	total: 2m 57s	remaining: 47.2s
790:	learn: 0.4826581	total: 2m 57s	remaining: 46.9s
791:	learn: 0.4826146	total: 2m 57s	remaining: 46.7s
792:	learn: 0.4825040	total: 2m 58s	remaining: 46.5s
793:	learn: 0.4822995	total: 2m 58s	remaining: 4

932:	learn: 0.4687614	total: 3m 29s	remaining: 15s
933:	learn: 0.4685957	total: 3m 29s	remaining: 14.8s
934:	learn: 0.4684594	total: 3m 29s	remaining: 14.6s
935:	learn: 0.4683146	total: 3m 30s	remaining: 14.4s
936:	learn: 0.4682441	total: 3m 30s	remaining: 14.1s
937:	learn: 0.4681234	total: 3m 30s	remaining: 13.9s
938:	learn: 0.4680817	total: 3m 30s	remaining: 13.7s
939:	learn: 0.4679319	total: 3m 31s	remaining: 13.5s
940:	learn: 0.4678109	total: 3m 31s	remaining: 13.2s
941:	learn: 0.4677759	total: 3m 31s	remaining: 13s
942:	learn: 0.4677278	total: 3m 31s	remaining: 12.8s
943:	learn: 0.4675893	total: 3m 31s	remaining: 12.6s
944:	learn: 0.4674935	total: 3m 32s	remaining: 12.3s
945:	learn: 0.4673834	total: 3m 32s	remaining: 12.1s
946:	learn: 0.4672595	total: 3m 32s	remaining: 11.9s
947:	learn: 0.4671701	total: 3m 32s	remaining: 11.7s
948:	learn: 0.4671379	total: 3m 33s	remaining: 11.5s
949:	learn: 0.4671074	total: 3m 33s	remaining: 11.2s
950:	learn: 0.4669984	total: 3m 33s	remaining: 11s

In [17]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(cb_model_std, open('./models/cb_model_basic_std.sav', 'wb'))
pickle.dump(cb_model_mm, open('./models/cb_model_basic_mm.sav', 'wb'))
pickle.dump(cb_model_ma, open('./models/cb_model_basic_ma.sav', 'wb'))
pickle.dump(cb_model_rb, open('./models/cb_model_basic_rb.sav', 'wb'))

In [19]:
std_data.to_csv('./train_scaled/std_data.csv', sep=',', index = False)
mm_data.to_csv('./train_scaled/mm_data.csv', sep=',', index = False)
ma_data.to_csv('./train_scaled/ma_data.csv', sep=',', index = False)
rb_data.to_csv('./train_scaled/rb_data.csv', sep=',', index = False)

### 테스트 데이터 처리

 1. 만들어 놓은 스케일러를 이용하여 test data를 스케일링한다. 
 2. 스케일링 데이터가 학습되어 있는 각각의 catboost모델을 이용하여 submission 작성
 3. submission이용 하여 어떤 스케일러가 가장 좋은 효과를 가져오는지 확인

In [35]:
def test_scaling_func(df, scaler) :
    # type과 id를 제외하고 학습
    data_for_scaling = df.drop(['id', 'fiberID'], axis = 1)
    scaler.fit(data_for_scaling)
    # 학습후 변환
    train_scaled = scaler.transform(data_for_scaling)
    # 학습후 변환한 데이터를 다시 원래 데이터로 만들기
    result = pd.DataFrame(train_scaled, columns = data_for_scaling.columns)
    result = pd.concat([df[['id', "fiberID"]], result], axis=1)
    return result

In [49]:
def make_submission(df, model) :
    id_for_index = df['id']
    test = df[test_data.columns[1:]]
    predictions = model.predict_proba(test)
    result = pd.DataFrame(data=predictions, index=id_for_index, columns=model.classes_)
    sample = pd.read_csv("./data/ybigta_sdss_sample_submission.csv")
    col_order = sample.columns[1:]
    result = result[col_order]
    return result

#### test data std scaling 

In [37]:
test_std = test_scaling_func(test_data, std_scaler)
test_std.head()

,id,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,70379,182,-0.870063,-0.619290,-0.609685,-0.522020,-0.375044,-0.839221,-0.587143,-0.513053,...,-0.658581,-0.405495,-0.323081,-0.211362,-0.116865,-0.699178,-0.388791,-0.316019,-0.206943,-0.076074
1,863365,207,1.320172,1.101081,0.759003,0.648224,1.263489,1.403418,1.184981,0.845732,...,1.222216,1.261362,0.918690,0.851994,1.123089,1.510004,1.282254,0.955425,0.864559,1.332756
2,40381,496,-0.814886,-0.401583,-0.130772,0.216036,0.341605,-0.808656,-0.360695,-0.057394,...,-0.627631,-0.184567,0.116198,0.442876,0.396200,-0.641505,-0.179602,0.120546,0.462762,0.533912
3,1322598,441,-0.651300,-0.434917,-0.074234,0.094952,0.043065,-0.913915,-0.790649,-0.552192,...,-1.231846,-1.246469,-1.121601,-1.060231,-0.993433,-1.272864,-1.203491,-1.021976,-0.972189,-0.862126
4,1201659,320,-0.469026,0.115018,0.303706,0.470840,0.335283,-0.444288,0.125275,0.232318,...,-0.510648,-0.117125,-0.272156,-0.268284,-0.367325,-0.727264,-0.241594,-0.292976,-0.238276,-0.380618


#### test data mm scaling 

In [40]:
test_mm = test_scaling_func(test_data, mm_scaler)
test_mm.head()

,id,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,70379,182,0.361414,0.406330,0.422762,0.423359,0.500008,0.358575,0.380491,0.365746,...,0.279762,0.284297,0.359046,0.320648,0.350592,0.536511,0.473568,0.477422,0.504832,0.525087
1,863365,207,0.743365,0.658682,0.586365,0.556051,0.698717,0.721887,0.635072,0.528722,...,0.428532,0.429495,0.471794,0.395690,0.448134,0.856653,0.702366,0.625648,0.619362,0.685838
2,40381,496,0.371036,0.438264,0.480007,0.507046,0.586918,0.363526,0.413022,0.420399,...,0.282210,0.303542,0.398931,0.366819,0.390953,0.544869,0.502210,0.528317,0.576415,0.594688
3,1322598,441,0.399563,0.433375,0.486766,0.493316,0.550713,0.346474,0.351255,0.361051,...,0.234417,0.211040,0.286544,0.260743,0.281636,0.453376,0.362019,0.395121,0.423037,0.435397
4,1201659,320,0.431350,0.514042,0.531942,0.535938,0.586151,0.422555,0.482836,0.455148,...,0.291464,0.309417,0.363670,0.316631,0.330889,0.532441,0.493722,0.480109,0.501483,0.490338


#### test data ma scaling

In [43]:
test_ma = test_scaling_func(test_data, ma_scaler)
test_ma.head()

,id,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,70379,182,0.728710,0.728185,0.717536,0.722766,0.758269,0.726799,0.722577,0.706175,...,0.509597,0.555759,0.616489,0.555334,0.582060,0.752108,0.732914,0.718097,0.723432,0.737500
1,863365,207,0.890974,0.843726,0.797593,0.786562,0.854339,0.881544,0.836581,0.781676,...,0.610893,0.645884,0.683951,0.604452,0.644835,0.923332,0.848995,0.798057,0.787401,0.826353
2,40381,496,0.732798,0.742806,0.745548,0.763001,0.800287,0.728908,0.737145,0.731494,...,0.511264,0.567704,0.640354,0.585554,0.608035,0.756578,0.747446,0.745552,0.763414,0.775971
3,1322598,441,0.744917,0.740568,0.748855,0.756400,0.782783,0.721645,0.709485,0.704000,...,0.478722,0.510288,0.573107,0.516123,0.537682,0.707644,0.676320,0.673700,0.677747,0.687926
4,1201659,320,0.758421,0.777501,0.770962,0.776892,0.799916,0.754050,0.768408,0.747591,...,0.517564,0.571350,0.619255,0.552705,0.569380,0.749931,0.743139,0.719546,0.721562,0.718293


#### test data rb scaling

In [44]:
test_rb = test_scaling_func(test_data, rb_scaler)
test_rb.head()

,id,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,70379,182,-0.598381,-0.572875,-0.529665,-0.487193,-0.313612,-0.605908,-0.535775,-0.410293,...,-0.478931,-0.277238,-0.177549,-0.130816,-0.053845,-0.488770,-0.264409,-0.175338,-0.131408,-0.033588
1,863365,207,1.122751,0.996939,0.512923,0.375855,0.882856,1.216748,1.096244,0.625163,...,1.395863,1.110589,0.685653,0.551176,0.763171,1.590509,1.091788,0.695069,0.542857,0.858803
2,40381,496,-0.555022,-0.374221,-0.164856,0.057119,0.209691,-0.581067,-0.327230,-0.063060,...,-0.448079,-0.093293,0.127810,0.288785,0.284218,-0.434488,-0.094634,0.123527,0.290018,0.352794
3,1322598,441,-0.426473,-0.404638,-0.121789,-0.032180,-0.008305,-0.666615,-0.723192,-0.440119,...,-1.050366,-0.977434,-0.732631,-0.675244,-0.631423,-1.028722,-0.925607,-0.658623,-0.612956,-0.531494
4,1201659,320,-0.283238,0.097170,0.166104,0.245035,0.205074,-0.284935,0.120318,0.157714,...,-0.331470,-0.037141,-0.142150,-0.167323,-0.218875,-0.515204,-0.144946,-0.159563,-0.151126,-0.226494


#### sbmissions 생성 및 저장

In [50]:
make_submission(test_std, cb_model_std).to_csv('./result/std_cb.csv', sep=',')
make_submission(test_mm, cb_model_mm).to_csv('./result/mm_cb.csv', sep=',')
make_submission(test_ma, cb_model_ma).to_csv('./result/ma_cb.csv', sep=',')
make_submission(test_rb, cb_model_rb).to_csv('./result/rb_cb.csv', sep=',')

결과

1) standardscaler : 0.4778  
2) minmaxscaler : 1.7  
3) maxabsscaler : 1.6  
4) robustscaler : 0.4778  
standardscaler 와 robustscaler 가 비슷한 성능을 보인다. 
scaling 되지 않은 데이터와 비교해보자

In [51]:
cb_model = cb.CatBoostClassifier()
X = data[data.columns[2:]]
y = data["type"]

cb_model.fit(X, y)

0:	learn: 2.6926061	total: 221ms	remaining: 3m 41s
1:	learn: 2.5202741	total: 441ms	remaining: 3m 39s
2:	learn: 2.3874659	total: 656ms	remaining: 3m 38s
3:	learn: 2.2759944	total: 876ms	remaining: 3m 38s
4:	learn: 2.1808330	total: 1.1s	remaining: 3m 38s
5:	learn: 2.1018191	total: 1.31s	remaining: 3m 37s
6:	learn: 2.0292334	total: 1.53s	remaining: 3m 37s
7:	learn: 1.9664018	total: 1.75s	remaining: 3m 37s
8:	learn: 1.9070229	total: 1.98s	remaining: 3m 37s
9:	learn: 1.8539631	total: 2.19s	remaining: 3m 37s
10:	learn: 1.8056311	total: 2.41s	remaining: 3m 37s
11:	learn: 1.7610348	total: 2.63s	remaining: 3m 36s
12:	learn: 1.7203762	total: 2.85s	remaining: 3m 36s
13:	learn: 1.6810912	total: 3.07s	remaining: 3m 36s
14:	learn: 1.6462434	total: 3.29s	remaining: 3m 36s
15:	learn: 1.6127006	total: 3.51s	remaining: 3m 35s
16:	learn: 1.5813686	total: 3.73s	remaining: 3m 35s
17:	learn: 1.5521009	total: 3.95s	remaining: 3m 35s
18:	learn: 1.5242098	total: 4.17s	remaining: 3m 35s
19:	learn: 1.4974558	to

158:	learn: 0.7056424	total: 35.3s	remaining: 3m 6s
159:	learn: 0.7035312	total: 35.6s	remaining: 3m 6s
160:	learn: 0.7028174	total: 35.8s	remaining: 3m 6s
161:	learn: 0.7018386	total: 36s	remaining: 3m 6s
162:	learn: 0.7007755	total: 36.2s	remaining: 3m 6s
163:	learn: 0.6991323	total: 36.4s	remaining: 3m 5s
164:	learn: 0.6977699	total: 36.7s	remaining: 3m 5s
165:	learn: 0.6964583	total: 36.9s	remaining: 3m 5s
166:	learn: 0.6944045	total: 37.1s	remaining: 3m 5s
167:	learn: 0.6930004	total: 37.3s	remaining: 3m 4s
168:	learn: 0.6923678	total: 37.6s	remaining: 3m 4s
169:	learn: 0.6908246	total: 37.8s	remaining: 3m 4s
170:	learn: 0.6895657	total: 38s	remaining: 3m 4s
171:	learn: 0.6889565	total: 38.2s	remaining: 3m 4s
172:	learn: 0.6883252	total: 38.5s	remaining: 3m 3s
173:	learn: 0.6866889	total: 38.7s	remaining: 3m 3s
174:	learn: 0.6855205	total: 38.9s	remaining: 3m 3s
175:	learn: 0.6846985	total: 39.1s	remaining: 3m 3s
176:	learn: 0.6839130	total: 39.4s	remaining: 3m 3s
177:	learn: 0.68

315:	learn: 0.5859316	total: 1m 10s	remaining: 2m 31s
316:	learn: 0.5855986	total: 1m 10s	remaining: 2m 31s
317:	learn: 0.5851621	total: 1m 10s	remaining: 2m 31s
318:	learn: 0.5844929	total: 1m 10s	remaining: 2m 31s
319:	learn: 0.5840274	total: 1m 11s	remaining: 2m 30s
320:	learn: 0.5835821	total: 1m 11s	remaining: 2m 30s
321:	learn: 0.5831612	total: 1m 11s	remaining: 2m 30s
322:	learn: 0.5828091	total: 1m 11s	remaining: 2m 30s
323:	learn: 0.5826347	total: 1m 11s	remaining: 2m 30s
324:	learn: 0.5823956	total: 1m 12s	remaining: 2m 29s
325:	learn: 0.5818225	total: 1m 12s	remaining: 2m 29s
326:	learn: 0.5814535	total: 1m 12s	remaining: 2m 29s
327:	learn: 0.5811330	total: 1m 12s	remaining: 2m 29s
328:	learn: 0.5804552	total: 1m 13s	remaining: 2m 28s
329:	learn: 0.5799821	total: 1m 13s	remaining: 2m 28s
330:	learn: 0.5798264	total: 1m 13s	remaining: 2m 28s
331:	learn: 0.5796619	total: 1m 13s	remaining: 2m 28s
332:	learn: 0.5790001	total: 1m 13s	remaining: 2m 28s
333:	learn: 0.5786566	total:

468:	learn: 0.5349825	total: 1m 45s	remaining: 1m 59s
469:	learn: 0.5348650	total: 1m 45s	remaining: 1m 58s
470:	learn: 0.5345365	total: 1m 45s	remaining: 1m 58s
471:	learn: 0.5342611	total: 1m 46s	remaining: 1m 58s
472:	learn: 0.5341924	total: 1m 46s	remaining: 1m 58s
473:	learn: 0.5339054	total: 1m 46s	remaining: 1m 58s
474:	learn: 0.5337247	total: 1m 46s	remaining: 1m 58s
475:	learn: 0.5335812	total: 1m 47s	remaining: 1m 58s
476:	learn: 0.5334045	total: 1m 47s	remaining: 1m 57s
477:	learn: 0.5332452	total: 1m 47s	remaining: 1m 57s
478:	learn: 0.5329767	total: 1m 47s	remaining: 1m 57s
479:	learn: 0.5327849	total: 1m 48s	remaining: 1m 57s
480:	learn: 0.5324490	total: 1m 48s	remaining: 1m 56s
481:	learn: 0.5323277	total: 1m 48s	remaining: 1m 56s
482:	learn: 0.5320008	total: 1m 48s	remaining: 1m 56s
483:	learn: 0.5317377	total: 1m 49s	remaining: 1m 56s
484:	learn: 0.5313989	total: 1m 49s	remaining: 1m 56s
485:	learn: 0.5312999	total: 1m 49s	remaining: 1m 55s
486:	learn: 0.5310106	total:

621:	learn: 0.5046613	total: 2m 20s	remaining: 1m 25s
622:	learn: 0.5044601	total: 2m 20s	remaining: 1m 25s
623:	learn: 0.5042479	total: 2m 21s	remaining: 1m 25s
624:	learn: 0.5040263	total: 2m 21s	remaining: 1m 24s
625:	learn: 0.5039416	total: 2m 21s	remaining: 1m 24s
626:	learn: 0.5038295	total: 2m 21s	remaining: 1m 24s
627:	learn: 0.5037950	total: 2m 21s	remaining: 1m 24s
628:	learn: 0.5036981	total: 2m 22s	remaining: 1m 23s
629:	learn: 0.5036385	total: 2m 22s	remaining: 1m 23s
630:	learn: 0.5035196	total: 2m 22s	remaining: 1m 23s
631:	learn: 0.5034268	total: 2m 22s	remaining: 1m 23s
632:	learn: 0.5032761	total: 2m 23s	remaining: 1m 22s
633:	learn: 0.5030513	total: 2m 23s	remaining: 1m 22s
634:	learn: 0.5029726	total: 2m 23s	remaining: 1m 22s
635:	learn: 0.5027998	total: 2m 23s	remaining: 1m 22s
636:	learn: 0.5025954	total: 2m 24s	remaining: 1m 22s
637:	learn: 0.5024412	total: 2m 24s	remaining: 1m 21s
638:	learn: 0.5022277	total: 2m 24s	remaining: 1m 21s
639:	learn: 0.5021102	total:

775:	learn: 0.4844636	total: 2m 56s	remaining: 51s
776:	learn: 0.4843816	total: 2m 56s	remaining: 50.8s
777:	learn: 0.4842880	total: 2m 57s	remaining: 50.5s
778:	learn: 0.4841908	total: 2m 57s	remaining: 50.3s
779:	learn: 0.4839999	total: 2m 57s	remaining: 50.1s
780:	learn: 0.4839210	total: 2m 57s	remaining: 49.9s
781:	learn: 0.4837999	total: 2m 58s	remaining: 49.6s
782:	learn: 0.4836835	total: 2m 58s	remaining: 49.4s
783:	learn: 0.4835093	total: 2m 58s	remaining: 49.2s
784:	learn: 0.4833411	total: 2m 58s	remaining: 48.9s
785:	learn: 0.4832595	total: 2m 58s	remaining: 48.7s
786:	learn: 0.4831897	total: 2m 59s	remaining: 48.5s
787:	learn: 0.4831119	total: 2m 59s	remaining: 48.3s
788:	learn: 0.4829805	total: 2m 59s	remaining: 48s
789:	learn: 0.4828267	total: 2m 59s	remaining: 47.8s
790:	learn: 0.4826581	total: 3m	remaining: 47.6s
791:	learn: 0.4826146	total: 3m	remaining: 47.4s
792:	learn: 0.4825040	total: 3m	remaining: 47.1s
793:	learn: 0.4822995	total: 3m	remaining: 46.9s
794:	learn: 0

932:	learn: 0.4687614	total: 3m 34s	remaining: 15.4s
933:	learn: 0.4685957	total: 3m 34s	remaining: 15.2s
934:	learn: 0.4684594	total: 3m 34s	remaining: 14.9s
935:	learn: 0.4683146	total: 3m 34s	remaining: 14.7s
936:	learn: 0.4682441	total: 3m 35s	remaining: 14.5s
937:	learn: 0.4681234	total: 3m 35s	remaining: 14.2s
938:	learn: 0.4680817	total: 3m 35s	remaining: 14s
939:	learn: 0.4679319	total: 3m 35s	remaining: 13.8s
940:	learn: 0.4678109	total: 3m 36s	remaining: 13.5s
941:	learn: 0.4677759	total: 3m 36s	remaining: 13.3s
942:	learn: 0.4677278	total: 3m 36s	remaining: 13.1s
943:	learn: 0.4675893	total: 3m 36s	remaining: 12.9s
944:	learn: 0.4674935	total: 3m 36s	remaining: 12.6s
945:	learn: 0.4673834	total: 3m 37s	remaining: 12.4s
946:	learn: 0.4672595	total: 3m 37s	remaining: 12.2s
947:	learn: 0.4671701	total: 3m 37s	remaining: 11.9s
948:	learn: 0.4671379	total: 3m 37s	remaining: 11.7s
949:	learn: 0.4671074	total: 3m 38s	remaining: 11.5s
950:	learn: 0.4669984	total: 3m 38s	remaining: 1

In [52]:
make_submission(test_data, cb_model).to_csv('./result/cb.csv', sep=',')

결과 
아무런 스케일링을 하지않은 데이터로 학습시킨 모델에서의 결과가 0.47726으로 가장 높게 나왔다. 
스케일링이 학습에 악영향을 미친다는 것을 확인할 수 있다